<a href="https://colab.research.google.com/github/rktreddy/Deep-Learning-for-Computer-Vision/blob/main/notebooks/health_analyzer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
medical_condition = pd.read_csv('medical_conditions_dataset.csv')
medical_condition.head()

In [ ]:
df = medical_condition.copy()
df['gender_smoking'] = df['gender'] + ' - ' + df['smoking_status']
df.head()

In [ ]:
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy='mean')

df[['age', 'bmi']] = mean_imputer.fit_transform(df[['age', 'bmi']])
print(df[["age", "bmi"]].isna().sum())

df['blood_pressure'] = df.groupby("gender_smoking")['blood_pressure'].transform(lambda x: x.fillna(x.mean()))
print(df[['blood_pressure']].isna().sum())

df['glucose_levels'] = df.groupby("gender_smoking")['glucose_levels'].transform(lambda x: x.fillna(x.mean()))
print(df[['glucose_levels']].isna().sum())

In [ ]:
df.drop(columns=["id", "full_name", "gender_smoking"], inplace=True)
df.head()

Handiling imbalanced data

In [1]:
X = df.drop(columns=["condition"])
y = df["condition"]

NameError: name 'df' is not defined

Feature Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

# onehot_encoder = OneHotEncoder()
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
X_ohe = onehot_encoder.fit_transform(X[categorical_columns])
# df_ohe.head()
X = pd.concat([X, X_ohe], axis=1)
X.drop(columns=categorical_columns, inplace=True)
X.head()

oversamopling using SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

PyTorch

In [ ]:
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(4, 8)
        self.act = nn.ReLU()
        self.output = nn.Linear(8, 3)

    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        return x